In [4]:
# Import modules
import sys
import os

# Add the parent directory of 'src' to the system path
sys.path.append(os.path.abspath("../"))  # if notebook is in /notebooks/
sys.path.append(os.path.abspath("./"))   # safe fallback

from src.data_loader import get_stock_data
from src.mc_simulation import monte_carlo_simulation, plot_simulation
from src.var_calc import historical_var, parametric_var
from src.lstm_model import train_lstm_model
from src.portfolio_opt import simulate_portfolios, plot_efficient_frontier

import numpy as np

# Load data
data = get_stock_data('AAPL')
returns = data['log_returns']
mu, sigma = returns.mean(), returns.std()

# Monte Carlo
S0 = data['Close'].iloc[-1]
sim = monte_carlo_simulation(S0, mu, sigma)
plot_simulation(sim)

# VaR
print("Historical VaR (95%):", historical_var(returns))
print("Parametric VaR (95%):", parametric_var(returns))

# LSTM
import matplotlib.pyplot as plt

series = data['Close'].values
model, scaler = train_lstm_model(series)

# Prediction
window = 60
input_seq = scaler.transform(series[-window:].reshape(-1,1)).reshape(1, window, 1)
predicted = model.predict(input_seq)
predicted_price = scaler.inverse_transform(predicted)[0][0]

print("Predicted next day price:", predicted_price)

# Portfolio Optimization
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
log_ret_df = []

for t in tickers:
    d = get_stock_data(t)
    log_ret_df.append(d['log_returns'])

import pandas as pd
log_returns_df = pd.concat(log_ret_df, axis=1)
log_returns_df.columns = tickers

results, weights = simulate_portfolios(log_returns_df)
plot_efficient_frontier(results)


ModuleNotFoundError: No module named 'tensorflow'